In [0]:
%run /Workspace/Users/surajss.3110@gmail.com/Learning/PySpark/MasterNoteBooks/INC_MasterNoteBook

In [0]:
inc = load_INC()
display(inc)

In [0]:
# 1. Dynamic SLA Breach Classification Create a logic that adjusts SLA breach thresholds based on ticket priority and business hours.

def SLAThresholds(Priority,Resolved_Timing_In_Hours):
    thresholds = {
        "1-Critical": 1,
        "2-High": 4,
        "Medium": 16,
        "Low": 24
    }
    limit = thresholds.get(Priority, 24)  # default to 24 if unknown
    return "SLA Breached" if Resolved_Timing_In_Hours > limit else "SLA Not Breached"
 
def is_weekend(timestamp):
    day = timestamp.weekday()  # Monday = 0, Sunday = 6
    return "Weekend" if day >= 5 else "Weekday"

udf1= udf(SLAThresholds,StringType())    
udf2= udf(is_weekend,StringType())    

df2a = (inc
        .withColumn("Opened",expr("try_to_timestamp(Opened,'MM/dd/yyyy HH:mm:ss')"))
        .withColumn("Resolved",expr("try_to_timestamp(Resolved,'MM/dd/yyyy HH:mm:ss')"))
        .na.drop(subset=["Opened","Resolved"])
        .withColumn("Resolved_Timing_in_Hours",(unix_timestamp("Resolved") - unix_timestamp("Opened"))/ 3600)
        .withColumn("Priority",udf1("Priority","Resolved_Timing_in_Hours"))
        .withColumn("Resolved_DayType", udf2("Resolved"))
        .select("Issue","Opened","Resolved","Resolved_Timing_in_Hours","Resolved_DayType","Priority")
        )
display(df2a)        




        

In [0]:
from pyspark.sql.functions import udf, expr, unix_timestamp
from pyspark.sql.types import StringType
from datetime import datetime, timedelta

# 1. Define the SLA classification function
def classify_sla(priority, opened, resolved):
    if not opened or not resolved:
        return "Unknown"

    # Convert to datetime if needed
    if isinstance(opened, str):
        opened = datetime.strptime(opened, "%Y-%m-%d %H:%M:%S")
    if isinstance(resolved, str):
        resolved = datetime.strptime(resolved, "%Y-%m-%d %H:%M:%S")

    # Calculate business hours (excluding weekends)
    current = opened
    total_hours = 0
    while current < resolved:
        if current.weekday() < 5:  # Monday to Friday
            total_hours += 1
        current += timedelta(hours=1)

    # Apply SLA thresholds
    if priority == "1-Critical":
        return "SLA Breached" if total_hours > 1 else "SLA Not Breached"
    elif priority == "2-High":
        return "SLA Breached" if total_hours > 6 else "SLA Not Breached"
    elif priority == "Medium":
        return "SLA Breached" if total_hours > 48 else "SLA Not Breached"  # 2 business days
    elif priority == "Low":
        return "SLA Breached" if total_hours > 96 else "SLA Not Breached"  # 4 business days
    else:
        return "Unknown"

# 2. Register the function as a UDF
sla_udf = udf(classify_sla, StringType())

# 3. Apply to your DataFrame
df_sla = (inc
    .withColumn("Opened", expr("try_to_timestamp(Opened,'MM/dd/yyyy HH:mm:ss')"))
    .withColumn("Resolved", expr("try_to_timestamp(Resolved,'MM/dd/yyyy HH:mm:ss')"))
    .na.drop(subset=["Opened", "Resolved"])
    .withColumn("Resolved_Timing_in_Hours", (unix_timestamp("Resolved") - unix_timestamp("Opened")) / 3600)
    .withColumn("SLA_Status", sla_udf("Priority", "Opened", "Resolved"))
    .select("Issue", "Opened", "Resolved", "Priority", "Resolved_Timing_in_Hours", "SLA_Status")
)

display(df_sla)


Databricks visualization. Run in Databricks to view.

Databricks data profile. Run in Databricks to view.